## Matching on DBLP ACM Dataset

### About the Dataset:
* This data set was taken from the Benchmark datasets for entity resolution web page. 
* It contains bibliographic data, with 4 attributes: title, authors, venue, year. 
* There are 3 CSV files in this zip archive.

### Task: Measure how accurate the bipartite matching algorithm is using the datasets and the ground truth presented in the data

Some assumptions and notes:
* We are checking string similarity using the `titles` column of the 2 datasets

* Added `encoded=latin-1` for pandas errors on file reading in the function `convert_df`

## Steps

#### 1. We'll include the bipartite matching algorithm components from our previous findings. We can try out different string matching techniques. 

#### A good starting point would be to see how `editDistance` is working in this context, so we'll include the function below.

In [1]:
import datetime
import textdistance
import editdistance

def editDistance(str1, str2, m, n): 
  
    # If first string is empty, the only option is to 
    # insert all characters of second string into first 
    if m == 0: 
         return n 
  
    # If second string is empty, the only option is to 
    # remove all characters of first string 
    if n == 0: 
        return m 
  
    # If last characters of two strings are same, nothing 
    # much to do. Ignore last characters and get count for 
    # remaining strings. 
    if str1[m-1]== str2[n-1]: 
        return editDistance(str1, str2, m-1, n-1) 
  
    # If last characters are not same, consider all three 
    # operations on last character of first string, recursively 
    # compute minimum cost for all three operations and take 
    # minimum of three values. 
    return 1 + min(editDistance(str1, str2, m, n-1),    # Insert 
                   editDistance(str1, str2, m-1, n),    # Remove 
                   editDistance(str1, str2, m-1, n-1)    # Replace 
                   ) 


def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [2]:
import pandas as pd
import networkx as nx

"""

Transforms the given file to a pandas dataframe object if it was not one already
Assumption: Assumes that the data starts from the 1st row of given file, does not use seperators such as "," or ";"

Input: 2 variables that will be measured for similarity
Output: The desired similarity metric result
"""
def similarity_edit(x,y):
    return editDistance(x,y,len(x),len(y))



"""

Transforms the given file to a pandas dataframe object if it was not one already
Assumption: Assumes that the data starts from the 1st row of given file, does not use seperators such as "," or ";"

Input: Any file
Output: A pandas dataframe object
"""
def convert_df(file):
    if isinstance(file, pd.DataFrame):
        return file
    else:
        df = pd.read_csv(file, encoding='latin-1')
        return df
"""

Calculates maximum weight for the matching

Input: keys from 2 tables
Output: weight for each matching to be used in the weight part of constructing the graph
"""
def calc_max_weight(key1, key2):
    weight = textdistance.jaccard(key1,key2) #this library's implementation is slower than jaccard_similarity()
    return weight

"""

Calculates minimum weight for the matching

Input: keys from 2 tables
Output: weight for each matching to be used in the weight part of constructing the graph
"""
def calc_min_weight(key1, key2):
    weight = (-1)/(1+textdistance.jaccard(key1,key2)) #this library's implementation is slower than jaccard_similarity()
    return weight

"""

Calculates maximum weight for the matching

Input: keys from 2 tables
Output: weight for each matching to be used in the weight part of constructing the graph
"""
def calc_max_weight_edit(key1, key2):
    weight = (1)/(1+editdistance.eval(key1,key2))
    return weight

"""

Calculates minimum weight for the matching

Input: keys from 2 tables
Output: weight for each matching to be used in the weight part of constructing the graph
"""
def calc_min_weight_edit(key1, key2):
    weight = (-1)/(1+editdistance.eval(key1,key2))
    return weight



"""

Converts the dataframe into dictionary for better accuracy matching of pairs. 
Assumption: The data has headers in the first row (description of what that column describes)

Input: Any file
Output: A dictionary in the form col1:col2 matching
"""
def make_dict(file):
    V = list(file.to_dict('list').values())
    keys = V[0]
    values = zip(*V[1:])
    table = dict(zip(keys,values))
    return table

"""

Constructs a maximal bipartite graph of the given two tables

Input: Any 2 files in any format
Output: A Bipartite Graph with Maximal Weights
"""
def updated_maximal_construct_graph(file_a, file_b):
    table_a_unprocessed = convert_df(file_a)
    table_b_unprocessed = convert_df(file_b)
    bipartite_graph = nx.Graph()
    
    table_a = make_dict(table_a_unprocessed)
    table_b = make_dict(table_b_unprocessed)
    
    i=0
    
    for key1, val1 in table_a.items():
       # print(val1)
        id1 = str(key1) + '_' + str(val1) + '_1'
        for key2, val2 in table_b.items():
            i+=1
            if i%100000 == 0:
                print(str(round(100*i/len(table_a)/len(table_b),2))+'% complete')
            #add value to identifier to distinguish two entries with different values
            id2 = str(key2) + '_' + str(val2) + '_2' 
            bipartite_graph.add_edge(id1, id2, weight=calc_max_weight_edit(val1, val2))
            #edit distance and weight should be inv. prop.
            #also adding 1 to denom. to prevent divide by 0
            # add 1,2 to distinguish two key-value tuples belonging to different tables
    return bipartite_graph



"""

Constructs a maximal bipartite graph of the given two tables

Input: Any 2 files in any format
Output: A Bipartite Graph with Minimal Weights
"""
def updated_minimal_construct_graph(file_a, file_b):
    table_a_unprocessed = convert_df(file_a)
    table_b_unprocessed = convert_df(file_a)
    bipartite_graph = nx.Graph()
    
    table_a = make_dict(table_a_unprocessed)
    table_b = make_dict(table_b_unprocessed)
    
    i=0
    
    for key1, val1 in table_a.items():
        id1 = str(key1) + '_' + str(val1) + '_1'
        for key2, val2 in table_b.items():
            i+=1
            if i%100000 == 0:
                print(str(round(100*i/len(table_a)/len(table_b),2))+'% complete')
            #add value to identifier to distinguish two entries with different values
            id2 = str(key2) + '_' + str(val2) + '_2' 
            bipartite_graph.add_edge(id1, id2, weight=calc_min_weight(key1, key2)) 
            #edit distance and weight should be inv. prop.
            #also adding 1 to denom. to prevent divide by 0
            # add 1,2 to distinguish two key-value tuples belonging to different tables
    return bipartite_graph

bipartite_graph_maximal = updated_maximal_construct_graph("table_a.csv","table_b.csv")
#print(bipartite_graph_maximal.edges.data())
bipartite_graph_minimal = updated_minimal_construct_graph("table_a.csv", "table_b.csv")
bipartite_graph_minimal.edges.data()

EdgeDataView([("US_('300 M', 1)_1", "US_('300 M', 1)_2", {'weight': -0.5}), ("US_('300 M', 1)_1", "CN_('12 B', 2)_2", {'weight': -1.0}), ("US_('300 M', 1)_1", "CA_('50 M', 3)_2", {'weight': -1.0}), ("US_('300 M', 1)_1", "AU_('25 M', 4)_2", {'weight': -0.75}), ("US_('300 M', 1)_2", "CN_('12 B', 2)_1", {'weight': -1.0}), ("US_('300 M', 1)_2", "CA_('50 M', 3)_1", {'weight': -1.0}), ("US_('300 M', 1)_2", "AU_('25 M', 4)_1", {'weight': -0.75}), ("CN_('12 B', 2)_2", "CN_('12 B', 2)_1", {'weight': -0.5}), ("CN_('12 B', 2)_2", "CA_('50 M', 3)_1", {'weight': -0.75}), ("CN_('12 B', 2)_2", "AU_('25 M', 4)_1", {'weight': -1.0}), ("CA_('50 M', 3)_2", "CN_('12 B', 2)_1", {'weight': -0.75}), ("CA_('50 M', 3)_2", "CA_('50 M', 3)_1", {'weight': -0.5}), ("CA_('50 M', 3)_2", "AU_('25 M', 4)_1", {'weight': -0.75}), ("AU_('25 M', 4)_2", "CN_('12 B', 2)_1", {'weight': -1.0}), ("AU_('25 M', 4)_2", "CA_('50 M', 3)_1", {'weight': -0.75}), ("AU_('25 M', 4)_2", "AU_('25 M', 4)_1", {'weight': -0.5})])

In [3]:
#nx.algorithms.matching.max_weight_matching(bipartite_graph_maximal)
print(nx.algorithms.bipartite.matching.maximum_matching(bipartite_graph_minimal))

{"US_('300 M', 1)_1": "US_('300 M', 1)_2", "CN_('12 B', 2)_1": "CN_('12 B', 2)_2", "CA_('50 M', 3)_1": "CA_('50 M', 3)_2", "AU_('25 M', 4)_1": "AU_('25 M', 4)_2", "AU_('25 M', 4)_2": "AU_('25 M', 4)_1", "CN_('12 B', 2)_2": "CN_('12 B', 2)_1", "US_('300 M', 1)_2": "US_('300 M', 1)_1", "CA_('50 M', 3)_2": "CA_('50 M', 3)_1"}


### 2. Load the data for processing

In [4]:
# Sticking to the convention of table_a and table_b naming that we previously used for generalization purposes

table_a = convert_df("ACM.csv")

table_b = convert_df("DBLP2.csv")

### 3. Create a bipartite graph

In [5]:
now = datetime.datetime.now()
graph_maximal = updated_maximal_construct_graph(table_a, table_b)
timing_bg = (datetime.datetime.now()-now).total_seconds()
print("---- Timing for Graph Construction ----")
print(timing_bg,"seconds")
graph_maximal.number_of_edges()

1.67% complete
3.33% complete
5.0% complete
6.67% complete
8.33% complete
10.0% complete
11.66% complete
13.33% complete
15.0% complete
16.66% complete
18.33% complete
20.0% complete
21.66% complete
23.33% complete
25.0% complete
26.66% complete
28.33% complete
29.99% complete
31.66% complete
33.33% complete
34.99% complete
36.66% complete
38.33% complete
39.99% complete
41.66% complete
43.33% complete
44.99% complete
46.66% complete
48.32% complete
49.99% complete
51.66% complete
53.32% complete
54.99% complete
56.66% complete
58.32% complete
59.99% complete
61.66% complete
63.32% complete
64.99% complete
66.65% complete
68.32% complete
69.99% complete
71.65% complete
73.32% complete
74.99% complete
76.65% complete
78.32% complete
79.99% complete
81.65% complete
83.32% complete
84.98% complete
86.65% complete
88.32% complete
89.98% complete
91.65% complete
93.32% complete
94.98% complete
96.65% complete
98.32% complete
99.98% complete
---- Timing for Graph Construction ----
35.265077 

6001104

As observed, from the above output, there are 6 million edges. This creates a very heavy computation for the maximal matching algorithm. It is very likely that we will observe that the maximum matching algorithm will not halt given the very large number of edges. 

In [6]:
#print(nx.algorithms.matching.max_weight_matching(graph_maximal))

The problem is that the max_weight_matching operation is O(n^3). The above operation does not halt given the large size of the dataset.

## Update - 04.08.2020 Wednesday
## Solution: Set a treshold to only connect the vertices that have the potential to be viable matches

The below solution sets a treshold of 0.3 for the jaccard similarity. However, this could be easily adjusted according to the similarity metric that we are using.

In [7]:
"""

Constructs a maximal bipartite graph of the given two tables according to the treshold similarity.
The bipartite matching graph only includes those that have passed a certain similarity treshold.

Input: Any 2 files in any format
Output: A Bipartite Graph with Minimal Weights
"""
def treshold_updated_maximal_construct_graph(file_a, file_b, treshold_decimal):
    table_a_unprocessed = convert_df(file_a)
    table_b_unprocessed = convert_df(file_b)
    bipartite_graph = nx.Graph()
    
    table_a = make_dict(table_a_unprocessed)
    table_b = make_dict(table_b_unprocessed)
    
    i=0
    
    for key1, val1 in table_a.items():
       # print(val1)
        id1 = str(key1) + '_'+ '_1'
        for key2, val2 in table_b.items():
            i+=1
            if i%100000 == 0:
                print(str(round(100*i/len(table_a)/len(table_b),2))+'% complete')
            if calc_max_weight_edit(val1,val2) >= treshold_decimal:
                #add value to identifier to disitnguish two entries with different values
                id2 = str(key2) + '_' + '_2' 
                bipartite_graph.add_edge(id1, id2, weight=calc_max_weight(val1, val2))
                #edit distance and weight should be inv. prop.
                #also adding 1 to denom. to prevent divide by 0
                # add 1,2 to distinguish two key-value tuples belonging to different tables
            else:
                continue
            
    return bipartite_graph

In [8]:
now = datetime.datetime.now()
treshold_graph_maximal = treshold_updated_maximal_construct_graph(table_a, table_b, 0.3)
timing_tresh = (datetime.datetime.now()-now).total_seconds()
print("---- Timing for Graph Construction with Treshold Constraint ----")
print(timing_tresh,"seconds")
#treshold_graph_maximal.number_of_edges()

"""

Outputs the matching that has the maximal weight for each edge in the bipartite graph

Input: A Bipartite Graph
Output: A set of matchings. Ex: {('journals/vldb/MedjahedBBNE03__2', '775457__1')}
"""
now = datetime.datetime.now()
matching_set = nx.algorithms.matching.max_weight_matching(treshold_graph_maximal)
timing_match = (datetime.datetime.now()-now).total_seconds()
print("---- Timing for Matching (Done on the graph constructed with the treshold constraint) ----")
print(timing_match,"seconds")
treshold_graph_maximal.number_of_edges()

1.67% complete
3.33% complete
5.0% complete
6.67% complete
8.33% complete
10.0% complete
11.66% complete
13.33% complete
15.0% complete
16.66% complete
18.33% complete
20.0% complete
21.66% complete
23.33% complete
25.0% complete
26.66% complete
28.33% complete
29.99% complete
31.66% complete
33.33% complete
34.99% complete
36.66% complete
38.33% complete
39.99% complete
41.66% complete
43.33% complete
44.99% complete
46.66% complete
48.32% complete
49.99% complete
51.66% complete
53.32% complete
54.99% complete
56.66% complete
58.32% complete
59.99% complete
61.66% complete
63.32% complete
64.99% complete
66.65% complete
68.32% complete
69.99% complete
71.65% complete
73.32% complete
74.99% complete
76.65% complete
78.32% complete
79.99% complete
81.65% complete
83.32% complete
84.98% complete
86.65% complete
88.32% complete
89.98% complete
91.65% complete
93.32% complete
94.98% complete
96.65% complete
98.32% complete
99.98% complete
---- Timing for Graph Construction with Treshold C

1442

As observed above, the graph construction with a treshold constraint was able to speed up the process by **27.58851 seconds**. I believe that this improvement in scalability will prove even better using even larger datasets.

Most importantly, the matching algorithm halted now that we have less edges to do computations on. The matching algorithm averaged around a total of **2.65 seconds** (timing computation process: took an average result of a few runs of the above block) to do computations on **1442 edges**. The matching algorithm worked surprisingly faster now that it did computations on the edges that would actually matter and we skipped the edges that had a very low possibility of being similar. 

A question that I had in mind is whether the treshold that I have set (0.3) is able to approximately capture the number of matchings that are given in the perfect mapping. In order to investigate this, we can take a look at the perfect matching dataframe.

In [9]:
perfect_mapping = convert_df("DBLP-ACM_perfectMapping.csv")
row = perfect_mapping.shape[0]
print("Number of entries in the perfect mapping file:", row)
print("Difference between the entries in perfect mapping and the matching set:", len(perfect_mapping) - len(matching_set))

Number of entries in the perfect mapping file: 2224
Difference between the entries in perfect mapping and the matching set: 959


Interestingly, there definitely are less edges in the graph that I've constructed using the treshold. The perfect mapping file shows that there are **2224 rows** of matched data, whereas my algorithm is going to work on matching **1442 edges**, which only captures 65% of the matching already. It might be a good idea to lower the treshold to see if we can capture that extra **959 edges** that my functions could not capture so far, or go over the `treshold_updated_maximal_construct_graph` again to see if there are any mistakes in creating the graph according to a certain treshold.

In [10]:
"""lowered_tresh_graph = treshold_updated_maximal_construct_graph(table_a, table_b, 0.25)
lowered_tresh_matching = nx.algorithms.matching.max_weight_matching(lowered_tresh_graph)
print(len(lowered_tresh_matching))"""
# The above computation does not halt after it reaches 99% completion ?

'lowered_tresh_graph = treshold_updated_maximal_construct_graph(table_a, table_b, 0.25)\nlowered_tresh_matching = nx.algorithms.matching.max_weight_matching(lowered_tresh_graph)\nprint(len(lowered_tresh_matching))'

In [11]:
"""

Constructs a maximal bipartite graph of the given two tables according to the treshold similarity.
The bipartite matching graph only includes those that have passed a certain similarity treshold.

Input: A set of vertices that have been matched according to the weight of the edges. Matching should be of type tuples inside a set.
Output: A Bipartite Graph with Minimal Weights
"""
def retrieve(matching):
    res2 = list(matching)
    res_tuple = []
    for i in res2:

        if int(i[0].split("_")[-1]) == 1:
            idACM = i[0].split("_")[0]
            idDBLP = i[1].split("_")[0]
            res_tuple.append((idDBLP, idACM))
            
        if int(i[0].split("_")[-1]) == 2:
            idACM = i[1].split("_")[0]
            idDBLP = i[0].split("_")[0]
            res_tuple.append((idDBLP, idACM))
            
    return res_tuple

retrieve(matching_set)

[('journals/sigmod/Qian97', '248618'),
 ('journals/sigmod/NascimentoSP03', '945722'),
 ('conf/vldb/ShanmugasundaramSBCLPR00', '671695'),
 ('conf/vldb/Sarawagi99', '671500'),
 ('conf/vldb/ChakrabartiDAR97', '673659'),
 ('journals/sigmod/Rosenthal98', '273270'),
 ('journals/sigmod/Finkelstein95', '219777'),
 ('journals/tods/TaoSP03', '958943'),
 ('journals/tods/Snodgrass01a', '505055'),
 ('conf/sigmod/KarbhariRXD02', '564773'),
 ('conf/sigmod/Jamil99', '304192'),
 ('conf/vldb/Schnase00', '671685'),
 ('conf/vldb/LakshmiZ98', '671200'),
 ('conf/vldb/RossS97', '670993'),
 ('conf/vldb/MorimotoIM97', '673651'),
 ('journals/sigmod/DunhamH95a', '219727'),
 ('journals/tods/RaschidL96', '236714'),
 ('journals/sigmod/Aberer01a', '604274'),
 ('conf/vldb/BliujuteJSS98', '671339'),
 ('conf/vldb/MiningGroup96', '673478'),
 ('conf/sigmod/ViglasN02', '564697'),
 ('conf/vldb/BlottKS98', '671333'),
 ('conf/vldb/GoldsteinR00', '671849'),
 ('conf/vldb/FangSGMU98', '671338'),
 ('conf/sigmod/SeshadriP97', '25

## Evaluating Accuracy

In [12]:
import csv

def eval_matching(matching):
    f = open('DBLP-ACM_perfectMapping.csv', 'r', encoding = "ISO-8859-1")
    reader = csv.reader(f, delimiter=',', quotechar='"')
    matches = set()
    proposed_matches = set()

    tp = set()
    fp = set()
    fn = set()
    tn = set()

    for row in reader:
        matches.add((row[0],row[1]))

    for m in matching:
     #   print(m)
        proposed_matches.add(m)

        if m in matches:
            tp.add(m)
        else:
            fp.add(m)

    for m in matches:
        if m not in proposed_matches:
            fn.add(m)

    prec = len(tp)/(len(tp) + len(fp))
    rec = len(tp)/(len(tp) + len(fn))

    return {'false positive': 1-prec, 
            'false negative': 1-rec,
            'accuracy': 2*(prec*rec)/(prec+rec)}


#prints out the accuracy
now = datetime.datetime.now()
out = eval_matching(retrieve(matching_set)) # retrieve() returns a list of tuples of DBLP2 ids and ACM ids.
timing = (datetime.datetime.now()-now).total_seconds()
print("----Accuracy----")
print(out)
print("---- Timing ----")
print(timing,"seconds")

----Accuracy----
{'false positive': 0.028458498023715362, 'false negative': 0.44764044943820225, 'accuracy': 0.704297994269341}
---- Timing ----
0.008047 seconds


After trying out a few different string similarity libraries (most promising ones being https://pypi.org/project/editdistance/0.3.1/ and https://pypi.org/project/textdistance/#description), I observed that the `jaccard_similarity` function that I wrote was surprisingly faster than the textdistance library implementation, so I've reverted the code to use `jaccard_similarity` instead again until we can find a better library for it. On the other hand, the `editDistance` library is a very fast implementation of the Levenshtein Distance in C. So I've written another function called `calc_max_weight_edit` and `calc_min_weight_edit` so that it is easier to switch between the similarity metrics in a more generalized code structure.

Another observation worthy of noting is that the Jaccard and the Levenshtein similarity metrics gave the same accuracy rate (70%).

### Investigating the issue of $|edges_{bipartite_graph}| < |rows_{perfect_mapping}|$

Coming back to the case of observing less edges than the rows that exist in the perfect mapping file, we can investigate this issue by checking which edges are absent according to the perfect mapping file and see if there is a pattern in these missing edges.

In [13]:
missing = []
"""for (idDBLP, idACM in perfect_mapping:
    if entry not in matching_set:
        print(entry)
        missing.append(entry)
print(missing)"""

mapped = set()
for index, row in perfect_mapping.iterrows():
    mapped.add((row["idDBLP"], row["idACM"]))
    
tp_inv = set()
fp_inv = set()

for i in matching_set:
    if i in mapped:
        tp_inv.add(i)
    else:
        fp_inv.add(i)
print(fp_inv)

{('journals/sigmod/Qian97__2', '248618__1'), ('945722__1', 'journals/sigmod/NascimentoSP03__2'), ('671695__1', 'conf/vldb/ShanmugasundaramSBCLPR00__2'), ('671500__1', 'conf/vldb/Sarawagi99__2'), ('673659__1', 'conf/vldb/ChakrabartiDAR97__2'), ('273270__1', 'journals/sigmod/Rosenthal98__2'), ('219777__1', 'journals/sigmod/Finkelstein95__2'), ('958943__1', 'journals/tods/TaoSP03__2'), ('journals/tods/Snodgrass01a__2', '505055__1'), ('conf/sigmod/KarbhariRXD02__2', '564773__1'), ('304192__1', 'conf/sigmod/Jamil99__2'), ('conf/vldb/Schnase00__2', '671685__1'), ('conf/vldb/LakshmiZ98__2', '671200__1'), ('670993__1', 'conf/vldb/RossS97__2'), ('conf/vldb/MorimotoIM97__2', '673651__1'), ('219727__1', 'journals/sigmod/DunhamH95a__2'), ('236714__1', 'journals/tods/RaschidL96__2'), ('604274__1', 'journals/sigmod/Aberer01a__2'), ('conf/vldb/BliujuteJSS98__2', '671339__1'), ('673478__1', 'conf/vldb/MiningGroup96__2'), ('conf/sigmod/ViglasN02__2', '564697__1'), ('conf/vldb/BlottKS98__2', '671333__1'

**Idea: Writing a generalized evaluation function for measuring whether the treshold can capture the number of matchings perfect mapping**